$\bf{Problem Statement:}$ Write a Python function that takes the name of a star and returns a galpy Orbit object (setup with the star's position, distance, proper motion, and radial velocity). Apply to: GJ 440, Lacaille 8760, Vega. Check that you are finding the right star.

In [5]:
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
from gaia_tools import query
from astropy.coordinates import SkyCoord

In [7]:
position = SkyCoord.from_name("GJ 440")
position

<SkyCoord (ICRS): (ra, dec) in deg
    (176.42881667, -64.84151667)>

In [8]:
position2 = SkyCoord.from_name('Lacaille 8760')
position2

<SkyCoord (ICRS): (ra, dec) in deg
    (319.31361994, -38.86735993)>

In [9]:
position3 = SkyCoord.from_name('Vega')
position3

<SkyCoord (ICRS): (ra, dec) in deg
    (279.23473479, 38.78368896)>

Write the SQL query statement

In [2]:
#sql = "SELECT * FROM gaiadr2.gaia_source WHERE ra BETWEEN"
#out = query.query()

Create galpy orbit, given a star's position, distance, proper motion, and radial velocity

In [3]:
# use natural unit: radius = 1 when circular velocity = 1. Everything is a fraction of these
R = 1 # radial distance
vR = 1 # radial velocity
vT = 1 # tangential velocity
z = 0.1 # height
vz = 0.1 # vertical velocity
phi = 0 # angle
# create orbit
o = Orbit(vxvv = [R,vR,vT,z,vz,phi])